# Rossmann Store Sales - Data Preprocessing Pipeline

This notebook provides a comprehensive data preprocessing pipeline for the Rossmann Store Sales dataset, including:
- Data loading and merging(optional)
- Data cleaning and missin value handling
- Feature engineering(creating new features to gain personalised insights)
- Data export to the form of graphs and or plots

## 1. Import Required Libraries

In [2]:
# Core DS libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os

# ML libraries
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

## 2. Load and Merge Rossmann Dataset

Load the training data and store information, then merge them for comprehensive analysis.

In [3]:
# Load train and store data
train = pd.read_csv('D:/BTD/rossman-eda-ml/data/raw/train.csv')
store = pd.read_csv('D:/BTD/rossman-eda-ml/data/raw/store.csv')

# Merge on 'Store' key
df = pd.merge(train, store, how='left', on='Store')

# Basic inspection
print(df.head())
print(df.info())
print(df.isna().sum())

C:\Users\Shahpure\AppData\Local\Temp\ipykernel_33276\1883215107.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('D:/BTD/rossman-eda-ml/data/raw/train.csv')


   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday StoreType Assortment  CompetitionDistance  \
0              1         c          a               1270.0   
1              1         a          a                570.0   
2              1         a          a              14130.0   
3              1         c          c                620.0   
4              1         a          a              29910.0   

   CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
0                        9.0                    2008.0       0   
1                   

## 4. Cleaning Data and datetime management

Analysing dataset, cleaning, filling in missing values, converting data types(date to datetime)....

In [12]:
# Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['WeekOfYear'] = df['Date'].dt.isocalendar().week

# Handle missing values
df['CompetitionDistance'].fillna(df['CompetitionDistance'].max(), inplace=True)
df['Promo2SinceYear'].fillna(0, inplace=True)
df['Promo2SinceWeek'].fillna(0, inplace=True)

# Encode categorical features
df['StoreType'] = df['StoreType'].astype('category').cat.codes
df['Assortment'] = df['Assortment'].astype('category').cat.codes
df['StateHoliday'] = df['StateHoliday'].astype('category').cat.codes

# Remove closed store days
df = df[df['Open'] != 0]

# Basic check again
print(df.info())
print(df.isna().sum())

# Save processed data
df.to_csv('D:/BTD/rossman-eda-ml/data/processed/train_processed.csv', index=False)

C:\Users\Shahpure\AppData\Local\Temp\ipykernel_33276\988305249.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['CompetitionDistance'].fillna(df['CompetitionDistance'].max(), inplace=True)
C:\Users\Shahpure\AppData\Local\Temp\ipykernel_33276\988305249.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

<class 'pandas.core.frame.DataFrame'>
Index: 844392 entries, 0 to 1017190
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      844392 non-null  int64         
 1   DayOfWeek                  844392 non-null  int32         
 2   Date                       844392 non-null  datetime64[ns]
 3   Sales                      844392 non-null  int64         
 4   Customers                  844392 non-null  int64         
 5   Open                       844392 non-null  int64         
 6   Promo                      844392 non-null  int64         
 7   StateHoliday               844392 non-null  int8          
 8   SchoolHoliday              844392 non-null  int64         
 9   StoreType                  844392 non-null  int8          
 10  Assortment                 844392 non-null  int8          
 11  CompetitionDistance        844392 non-null  float64     

In [16]:
processed = pd.read_csv("D:/BTD/rossman-eda-ml/data/processed/train_processed.csv")
processed.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance               0
CompetitionOpenSinceMonth    268619
CompetitionOpenSinceYear     268619
Promo2                            0
Promo2SinceWeek                   0
Promo2SinceYear                   0
PromoInterval                423307
Year                              0
Month                             0
WeekOfYear                        0
dtype: int64

## 4. Advanced Feature Engineering

Creating meaningful business-driven features that provide deep insights into sales patterns and customer behavior.

In [ ]:
# ============================================================================
# ADVANCED FEATURE ENGINEERING FOR BUSINESS INSIGHTS
# ============================================================================

print("🔧 ADVANCED FEATURE ENGINEERING")
print("=" * 50)

# Ensure Date column is datetime
df['Date'] = pd.to_datetime(df['Date'])

# ============================================================================
# FEATURE 1: SALES VELOCITY METRICS
# ============================================================================
print("\n1️⃣ SALES VELOCITY METRICS")
print("-" * 35)

# Sales per customer (efficiency metric)
df['SalesPerCustomer'] = df['Sales'] / (df['Customers'] + 1)  # +1 to avoid division by zero

# Customer conversion rate (customers vs potential based on day patterns)
df['CustomerDensity'] = df['Customers'] / df['Sales'].clip(lower=1)  # How many customers per euro

print("✅ Created Features:")
print("• SalesPerCustomer: Average spending per customer")
print("• CustomerDensity: Customer efficiency metric")
print(f"• Average Sales per Customer: €{df['SalesPerCustomer'].mean():.2f}")

# ============================================================================
# FEATURE 2: TEMPORAL BUSINESS PATTERNS
# ============================================================================
print("\n2️⃣ TEMPORAL BUSINESS PATTERNS")
print("-" * 40)

# Extract comprehensive date features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['DayOfMonth'] = df['Date'].dt.day
df['WeekOfYear'] = df['Date'].dt.isocalendar().week
df['Quarter'] = df['Date'].dt.quarter

# Business-critical temporal features
df['IsWeekend'] = (df['DayOfWeek'] >= 5).astype(int)  # Friday=5, Saturday=6, Sunday=0
df['IsMonthStart'] = (df['DayOfMonth'] <= 7).astype(int)  # First week of month
df['IsMonthEnd'] = (df['DayOfMonth'] >= 25).astype(int)  # Last week of month
df['IsQuarterEnd'] = df['Month'].isin([3, 6, 9, 12]).astype(int)  # Quarter end months

# Seasonal patterns
df['Season'] = df['Month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Autumn', 10: 'Autumn', 11: 'Autumn'
})

print("✅ Created Features:")
print("• IsWeekend: Weekend sales indicator")
print("• IsMonthStart/End: Month timing patterns")
print("• Season: Seasonal sales patterns")
print("• IsQuarterEnd: Quarter-end business cycles")

# ============================================================================
# FEATURE 3: COMPETITION INTELLIGENCE
# ============================================================================
print("\n3️⃣ COMPETITION INTELLIGENCE")
print("-" * 40)

# Competition duration (how long has competition been active)
df['CompetitionDuration'] = (df['Year'] - df['CompetitionOpenSinceYear']) * 12 + \
                           (df['Month'] - df['CompetitionOpenSinceMonth'])
df['CompetitionDuration'] = df['CompetitionDuration'].clip(lower=0)

# Competition proximity categories
df['CompetitionProximity'] = pd.cut(
    df['CompetitionDistance'],
    bins=[0, 500, 1000, 5000, float('inf')],
    labels=['Very_Close', 'Close', 'Moderate', 'Far']
)

# Competition pressure score (combination of distance and duration)
df['CompetitionPressure'] = (df['CompetitionDuration'] + 1) / (df['CompetitionDistance'] + 1) * 1000

print("✅ Created Features:")
print("• CompetitionDuration: Months since competition opened")
print("• CompetitionProximity: Distance-based competition categories")
print("• CompetitionPressure: Combined competition intensity score")
print(f"• Average Competition Duration: {df['CompetitionDuration'].mean():.1f} months")

# ============================================================================
# FEATURE 4: PROMOTION EFFECTIVENESS
# ============================================================================
print("\n4️⃣ PROMOTION EFFECTIVENESS")
print("-" * 40)

# Promo2 duration (how long store has been in extended promotion)
df['Promo2Duration'] = np.where(
    df['Promo2'] == 1,
    (df['Year'] - df['Promo2SinceYear']) * 52 + (df['WeekOfYear'] - df['Promo2SinceWeek']),
    0
)
df['Promo2Duration'] = df['Promo2Duration'].clip(lower=0)

# Promotion synergy effects
df['PromoSynergy'] = df['Promo'] * df['Promo2']  # Both promotions active
df['PromoWithHoliday'] = df['Promo'] * (df['StateHoliday'] != 0).astype(int)
df['PromoWithSchoolHoliday'] = df['Promo'] * df['SchoolHoliday']

# Promotion intensity score
df['PromotionIntensity'] = df['Promo'] + df['Promo2'] + df['PromoWithHoliday'] + df['PromoWithSchoolHoliday']

print("✅ Created Features:")
print("• Promo2Duration: Weeks in extended promotion")
print("• PromoSynergy: Multiple promotions active")
print("• PromoWithHoliday: Promotion during holidays")
print("• PromotionIntensity: Overall promotion pressure")
print(f"• Average Promo2 Duration: {df['Promo2Duration'].mean():.1f} weeks")

# ============================================================================
# FEATURE 5: STORE PERFORMANCE METRICS
# ============================================================================
print("\n5️⃣ STORE PERFORMANCE METRICS")
print("-" * 40)

# Store-level aggregated features
store_metrics = df.groupby('Store').agg({
    'Sales': ['mean', 'std', 'max'],
    'Customers': 'mean',
    'SalesPerCustomer': 'mean'
}).round(2)

# Flatten column names
store_metrics.columns = ['StoreAvgSales', 'StoreSalesStd', 'StoreMaxSales', 
                        'StoreAvgCustomers', 'StoreAvgSalesPerCustomer']

# Merge back to main dataframe
df = df.merge(store_metrics, left_on='Store', right_index=True, how='left')

# Performance relative to store average
df['SalesVsStoreAvg'] = df['Sales'] / df['StoreAvgSales']
df['IsAboveStoreAvg'] = (df['SalesVsStoreAvg'] > 1).astype(int)

# Store performance classification
df['StorePerformanceClass'] = pd.cut(
    df['StoreAvgSales'],
    bins=[0, 5000, 8000, 12000, float('inf')],
    labels=['Low', 'Medium', 'High', 'Premium']
)

print("✅ Created Features:")
print("• StoreAvgSales: Historical store performance")
print("• SalesVsStoreAvg: Daily performance vs store average")
print("• StorePerformanceClass: Store tier classification")
print("• IsAboveStoreAvg: Above-average performance indicator")

# ============================================================================
# FEATURE 6: CUSTOMER BEHAVIOR PATTERNS
# ============================================================================
print("\n6️⃣ CUSTOMER BEHAVIOR PATTERNS")
print("-" * 40)

# Sort by store and date for time series features
df = df.sort_values(['Store', 'Date'])

# Lag features for trend analysis
df['SalesLag1'] = df.groupby('Store')['Sales'].shift(1)
df['SalesLag7'] = df.groupby('Store')['Sales'].shift(7)  # Same day last week
df['CustomersLag1'] = df.groupby('Store')['Customers'].shift(1)

# Rolling averages for trend smoothing
df['SalesRollingMean7'] = df.groupby('Store')['Sales'].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
df['SalesRollingMean30'] = df.groupby('Store')['Sales'].rolling(window=30, min_periods=1).mean().reset_index(0, drop=True)

# Trend indicators
df['SalesTrend7'] = df['Sales'] - df['SalesRollingMean7']
df['CustomerTrend'] = df['Customers'] - df['CustomersLag1']

# Fill NaN values with appropriate defaults
df['SalesLag1'].fillna(df['Sales'].mean(), inplace=True)
df['SalesLag7'].fillna(df['Sales'].mean(), inplace=True)
df['CustomersLag1'].fillna(df['Customers'].mean(), inplace=True)
df['CustomerTrend'].fillna(0, inplace=True)

print("✅ Created Features:")
print("• SalesLag1/7: Previous day and week sales")
print("• SalesRollingMean7/30: Trend smoothing")
print("• SalesTrend7: Short-term sales momentum")
print("• CustomerTrend: Customer flow changes")

# ============================================================================
# FEATURE SUMMARY
# ============================================================================
print("\n📊 FEATURE ENGINEERING SUMMARY")
print("-" * 45)

new_features = [
    'SalesPerCustomer', 'CustomerDensity', 'IsWeekend', 'IsMonthStart', 'IsMonthEnd',
    'Season', 'CompetitionDuration', 'CompetitionProximity', 'CompetitionPressure',
    'Promo2Duration', 'PromoSynergy', 'PromotionIntensity', 'StoreAvgSales',
    'SalesVsStoreAvg', 'StorePerformanceClass', 'SalesLag1', 'SalesRollingMean7',
    'SalesTrend7', 'CustomerTrend'
]

print(f"Total new features created: {len(new_features)}")
print(f"Dataset shape after feature engineering: {df.shape}")

# Save the feature-engineered dataset
df.to_csv('D:/BTD/rossman-eda-ml/data/processed/train_features.csv', index=False)
print("\n✅ Feature-engineered dataset saved to 'data/processed/train_features.csv'")

# Display sample of new features
print("\n🔍 Sample of new features:")
sample_features = ['Store', 'Date', 'Sales', 'SalesPerCustomer', 'CompetitionPressure', 
                  'PromotionIntensity', 'StorePerformanceClass', 'SalesTrend7']
print(df[sample_features].head(10).to_string())

print("\n🎯 Ready for Exploratory Data Analysis!")